<a href="https://colab.research.google.com/github/Karthikard/ML/blob/main/20_hyperparatuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [8]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [9]:
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.3)

In [13]:
from sklearn.svm import SVC
model = SVC()

In [16]:
model=SVC(kernel='rbf' , C=30 , gamma='auto')
model.fit(X_train,y_train)

SVC(C=30, gamma='auto')

In [21]:
model.score(X_test,y_test)

0.9777777777777777

In [22]:
from sklearn.model_selection import cross_val_score

In [24]:
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [25]:
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [27]:
cross_val_score(SVC(kernel='rbf' , C=20,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [31]:
#instead of above ...easy method
import numpy as np
kernels = ['rbf' , 'linear']
c = [1,10,20]
avg_score = {}
for kval in kernels:
  for cval in c:
    cv_scores = cross_val_score(SVC(kernel = kval , C=cval , gamma='auto'),iris.data,iris.target , cv=5)
    avg_score[kval + '-' +str(cval)] = np.average(cv_scores)

avg_score

{'rbf-1': np.float64(0.9800000000000001),
 'rbf-10': np.float64(0.9800000000000001),
 'rbf-20': np.float64(0.9666666666666668),
 'linear-1': np.float64(0.9800000000000001),
 'linear-10': np.float64(0.9733333333333334),
 'linear-20': np.float64(0.9666666666666666)}

In [36]:
#another easy method
#grid search CV  : to find the best combination of parameters
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(gamma = 'auto') , {
    'C' : [1,10,20] ,
    'kernel': ['rbf','linear']
} , cv = 5 ,return_train_score = False)   #Only returns test scores (not training scores)
clf.fit(iris.data,iris.target)
clf.cv_results_ #dictionary that contains detailed results of each cross-validation combination. like mean_test_score,std_test_score,params,rank_test_score


{'mean_fit_time': array([0.00253601, 0.00200634, 0.00199618, 0.00179119, 0.00298839,
        0.00164261]),
 'std_fit_time': array([0.00067067, 0.00019775, 0.00039473, 0.000213  , 0.00212185,
        0.00011827]),
 'mean_score_time': array([0.00314608, 0.00375628, 0.00142035, 0.00128131, 0.00145936,
        0.00122175]),
 'std_score_time': array([2.04726206e-03, 4.19675392e-03, 1.38868178e-04, 6.66282294e-05,
        1.61896714e-04, 2.84682566e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'li

In [34]:
df=pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002100,0.000239,0.001582,0.000151,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001814,0.000203,0.001308,0.000049,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001919,0.000183,0.001458,0.000265,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001670,0.000086,0.001260,0.000096,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001853,0.000065,0.001282,0.000041,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001774,0.000130,0.001354,0.000296,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [35]:
df[['param_C' , 'param_kernel' , 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [37]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_run_sea

In [38]:
clf.best_score_

np.float64(0.9800000000000001)

In [39]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [41]:
clf.error_score

nan

Randomized Search CV (less computational cost compared to gridsearchCV)

In [48]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma='auto') , {
    'C' : [1,10,20],
    'kernel' : ['rbf' , 'linear']
    },
    cv =5 ,
    return_train_score=False ,
    n_iter=2)  #try nly 2 combination

rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)[['param_C' , 'param_kernel' , 'mean_test_score']]


,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,20,rbf,0.966667


In [49]:
#hyperparameter tuning
#finding best model for iris
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [58]:
model_params = {
    'svm' : {
        'model' : svm.SVC(gamma='auto'),
        'params' : {
            'C' : [1,10,20],
            'kernel' : ['rbf' , 'linear']
        }
    },

    'rf' : {
      'model' : RandomForestClassifier(),
      'params' : {
          'n_estimators' : [1,5,10]
      }
    },

    'lr' : {
      'model' : LogisticRegression(solver='liblinear'),
      'params' : {
          'C' : [1,5,10]
      }
    }

}




In [52]:
from sklearn.datasets import load_iris
iris = load_iris()

In [59]:
scores = []

for model_name , mp in model_params.items():
  clf = GridSearchCV(mp['model'] , mp['params'] , cv=5 , return_train_score=False)
  clf.fit(iris.data,iris.target)
  scores.append({
      'model' : model_name,
      'best_score' : clf.best_score_,
      'best_params' : clf.best_params_
  })

In [61]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_run_sea

In [60]:
dff = pd.DataFrame(scores , columns=['model' , 'best_score' , 'best_params'])
dff

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,rf,0.960000,{'n_estimators': 10}
2,lr,0.966667,{'C': 5}
